# <복습>
- 딥러닝 + cv2

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys
import os

file_list = os.listdir('opencv_DNN/googlenet/fig/')
print(file_list)

img_list = []
for i in file_list:
    img_list.append('opencv_DNN/googlenet/fig/' + i)

    
model = 'googlenet/bvlc_googlenet.caffemodel'
config = 'googlenet/deploy.prototxt'

net = cv2.dnn.readNet(model, config)

if net.empty():
    sys.exit('실패')
    
class_names = []
with open('googlenet/classification_classes_ILSVRC2012.txt', 'rt') as f:
    class_names = f.read().rstrip('\n').split('\n')

index = 0
while True:
    print(img_list[index])
    img = cv2.imread(img_list[index])
    
    if img is None:
        sys.exit('오류')
        break
        
    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123), 
                             swapRB = False)
    net.setInput(blob)
    prob = net.forward()
    
    out = prob.flatten()
    print(out[0:5])
    class_id = np.argmax(out)
    
    confidence = out[class_id]
    category = class_names[class_id]
    
    text = f'{category} ({confidence*100:.2f}%)'
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX,
               1, (100, 0, 230), 1, cv2.LINE_AA)
        
#     img = cv2.resize(img, (900, 600), interpolation = cv2.INTER_AREA)
    cv2.imshow('img', img)
    
    if cv2.waitKey(3000) == 27:
        break
    index += 1
    if index > len(file_list):
        index = 0
        
cv2.destroyAllWindows()

['apple1.png', 'apple2.png', 'beagle.jpg', 'fish.jpg', 'pineapple.jpg', 'scooter.jpg', 'space_shuttle.jpg']
opencv_DNN/googlenet/fig/apple1.png
[3.9514849e-09 1.8157695e-07 2.1860103e-07 2.7997571e-09 3.2794156e-09]


In [22]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys
import os

model = 'googlenet/face_detector/opencv_face_detector_uint8.pb'
config = 'googlenet/face_detector/opencv_face_detector.pbtxt'

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    cap.release()
    sys.exit('실패')
    
net = cv2.dnn.readNet(model, config)

if net.empty():
    sys.exit('오류')
    
while True:
    ret, frame = cap.read()
    
    if not ret:
        print('ret None')
        break
        
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123))
    net.setInput(blob)
    out = net.forward()  # out(1, 1, 200, 7)
    
    detect = out[0, 0, :, :]
    h, w = frame.shape[:2]
    
    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        
        if confidence > 0.5:
            x1 = int(detect[i, 3]*w)
            y1 = int(detect[i, 4]*h)
            x2 = int(detect[i, 5]*w)
            y2 = int(detect[i, 6]*h)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2),
                         (90, 100, 255), 2)
            
            text = f'Face : P{confidence:.2f}'
            cv2.putText(frame, text, (x1, y1-5), cv2.FONT_HERSHEY_COMPLEX,
                       1, (200, 100, 0), 1, cv2.LINE_AA)
            
        cv2.imshow('frame', frame)
        
        if cv2.waitKey(3000) == 27:
            break
            
cap.release()
cv2.destroyAllWindows()

## 히스토그램 역투영

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys
import os

src = cv2.imread('Ch_4_Histogram_modi/fig/green.png', cv2.IMREAD_REDUCED_COLOR_2)
src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

x, y, w, h = cv2.selectROI(src)
crop = src_hsv[y:y+h, x:x+w]
joint_hist = cv2.calcHist([crop], [0, 1], None, [180, 256],
                         [0, 180, 0, 256])
back_proj = cv2.calcBackProject([src_hsv], [0, 1], joint_hist,
                               [0, 180, 0, 256], 1)

dft = cv2.copyTo(src, back_proj)

cv2.imshow('src', src)
cv2.imshow('crop', crop)
cv2.imshow('joint_hist', joint_hist)
cv2.imshow('back_proj', back_proj)
cv2.imshow('dft', dft)

cv2.waitKey()
cv2.destroyAllWindows()

## 컨벌루션
- 평균 필터 제작
- blur함수는 적절하지 않다. 따라서 가우시안 필터 사용.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys

src = cv2.imread('Ch_5_filtering/fig/blue_eyes.png')

# 커널 제작
kernel3 = np.ones((3, 3), np.float32) / 9.0
kernel5 = np.ones((5, 5), np.float32) / 25.0
kernel9 = np.ones((9, 9), np.float32) / 81.0

# 커널 적용
dst3 = cv2.filter2D(src, -1, kernel3)
dst5 = cv2.filter2D(src, -1, kernel5)
dst9 = cv2.filter2D(src, -1, kernel9)

# 다른 방법 - blur()
dst_bulr = cv2.blur(src, (3, 3))

for ksize in (3, 5, 7, 9, 11, 13, 15, 3):
    dst = cv2.blur(src, (ksize, ksize))
    
    text = '{} * {}'.format(ksize, ksize)
    cv2.putText(dst, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX,
               1, (100, 255, 200), 1, cv2.LINE_AA)
    cv2.imshow('dst', dst)
    cv2.waitKey(2000)

# cv2.imshow('src', src)
# cv2.imshow('dst3', dst3)
# cv2.imshow('dst5', dst5)
# cv2.imshow('dst9', dst9)
# cv2.imshow('dst_bulr', dst_bulr)

# cv2.waitKey()
cv2.destroyAllWindows()

### 가우시안 필터
- 질감, 색상 유지하면서 노이즈 제거 가능

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys

src = cv2.imread('Ch_5_filtering/fig/blue_eyes.png')

dst_gaussian_1 = cv2.GaussianBlur(src, (0, 0), 1)
dst_gaussian_2 = cv2.GaussianBlur(src, (0, 0), 2)
dst_gaussian_half = cv2.GaussianBlur(src, (0, 0), 0.5)

dst_bulr = cv2.blur(src, (7, 7))


cv2.imshow('src', src)
cv2.imshow('dst_gaussian_1', dst_gaussian_1)
cv2.imshow('dst_gaussian_2', dst_gaussian_2)
cv2.imshow('dst_gaussian_half', dst_gaussian_half)
cv2.imshow('dst_bulr', dst_bulr)

cv2.waitKey()
cv2.destroyAllWindows()

## 샤프닝 필터

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys

src = cv2.imread('Ch_5_filtering/fig/blue_eyes.png', 0)
blur = src.copy()
blur = cv2.GaussianBlur(blur, (0, 0), 3)


# src * 2 - bulr
# dst_sharp = np.clip(src*2.0 - blur, 0, 255).astype(np.uint8)
dst_sharp2 = cv2.addWeighted(src, 2, blur, -1.3, 0)


cv2.imshow('src', src)
cv2.imshow('blur', blur)
cv2.imshow('dst_sharp', dst_sharp2)

cv2.waitKey()
cv2.destroyAllWindows()

color

In [47]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys

src = cv2.imread('Ch_5_filtering/fig/blue_eyes.png', 1)

src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(src_hsv)

gaussian_v = cv2.GaussianBlur(v, (0, 0), 2)
src_hsv_g = cv2.merge((h, s, gaussian_v))

dst = cv2.addWeighted(src_hsv, 2.2, src_hsv_g, -1.3, 0)
dst = cv2.cvtColor(dst, cv2.COLOR_HSV2BGR)


cv2.imshow('src', src)
cv2.imshow('src_hsv_g', src_hsv_g)
cv2.imshow('gaussian_v', gaussian_v)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

## 잡음제거 필터 (Median filter)
- 필터에 값이 있는게 아니라 영역을 지정하여 중간 값으로 대체한다

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys

src = cv2.imread('Ch_5_filtering/fig/blue_eyes.png', 0)
print(src.shape)

noise1 = np.random.choice((0, 255), src.shape[:2], p = (0.90, 0.10))
noise2 = np.random.choice((0, 255), src.shape[:2], p = (0.90, 0.10))

src_noise = src + noise1 - noise2
src_noise = np.clip(src_noise, 0, 255).astype(np.uint8)

dst = cv2.medianBlur(src_noise, 5)

cv2.imshow('src', src)
cv2.imshow('src_noise', src_noise)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

(553, 840)


## 양방향 필터
- 이미지에 대한 비선형, 에지 보존 및 노이즈 감소 스무딩 필터

In [42]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys

src = cv2.imread('Ch_5_filtering/fig/blue_eyes.png', 0)

dst_gaussian = cv2.GaussianBlur(src, (0, 0), 3)
dst_bilteral = cv2.bilateralFilter(src, -1, 10, 3)

cv2.imshow('src', src)
cv2.imshow('dst_gaussian', dst_gaussian)
cv2.imshow('dst_bilteral', dst_bilteral)

cv2.waitKey()
cv2.destroyAllWindows()

## Cartoon 이미지 만들기

In [47]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys

src = cv2.imread('Ch_5_filtering/fig/Son.jpg')

blur = cv2.GaussianBlur(src, (0, 0), 2)
edge = cv2.Canny(src, 10, 150)

# 구조화 요소 커널, 사각형 (3x3) 생성 ---①
k = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# 팽창 연산 적용 ---②
# edge = cv2.dilate(edge, k)
edge = 255 - edge

# print(edge.shape) # 흑백임
edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)  # BGR로 바꿔줌
dst = cv2.bitwise_and(blur, edge)

cartoon_img = cv2.stylization(src, sigma_s = 10, sigma_r=0.7)

cv2.imshow('src', src)
cv2.imshow('blur', blur)
cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.imshow('cartoon_img', cartoon_img)

cv2.waitKey()
cv2.destroyAllWindows()

In [44]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as matim
import ipywidgets as widgets

img = cv2.imread('Ch_5_filtering/fig/Son.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

@widgets.interact_manual(style = ["detail", "pencil", "cartoon"])
def app(style="pencil"):
    
    if style == "detail":
        @widgets.interact_manual(s = (0, 200, 1), r = (0, 1, 0.1))
        def dtail(s = 100, r = 0.5):
            detailImg = cv2.detailEnhance(img, sigma_s = s, sigma_r = r)
            plt.figure(figsize = (15, 8)) 
            plt.subplot(1,2,1)
            plt.imshow(img)
            plt.yticks([]), plt.xticks([]) # 눈금을 null 값으로 넣겠다. 
            plt.title("Original Image")
            plt.subplot(1,2,2)
            plt.imshow(detailImg)
            plt.title("Dtail Image")
            plt.yticks([]), plt.xticks([])
            plt.show()
            
    elif style == "pencil":
        def adjust_gamma(image, gamma = 1):
            invGamma = 1.0 / gamma 
            table = np.array([
                ((i/255)**invGamma) * 255 for i in np.arange(0, 256)
            ])
            lut_img = cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))
            return lut_img
        @widgets.interact_manual(ksize=(3, 25, 2), sigmaX = (1, 15, 2), gamma = (0, 1, 0.01))
        def pencil_art_image(ksize = 21, sigmaX = 9, gamma = 0.1):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_blur = cv2.GaussianBlur(gray, (ksize, ksize), sigmaX) 
            gray_blur_divide = cv2.divide(gray, gray_blur, scale = 256)
            pencil_sketch = adjust_gamma(gray_blur_divide, gamma = gamma) 

            plt.figure(figsize = (15, 8))
            plt.subplot(1, 2, 1)
            plt.yticks([]), plt.xticks([])
            plt.title("Original Image")
            plt.imshow(img)
            plt.subplot(1, 2, 2)
            plt.yticks([]), plt.xticks([])
            plt.title("Pencil Image")
            plt.imshow(pencil_sketch, cmap = "gray")
            plt.show()
            
    elif style == "cartoon":
        def edge_mask(img, ksize, block_size):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_median = cv2.medianBlur(gray, ksize)
            edges = cv2.adaptiveThreshold(gray_median, 255, 
                cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, ksize)
            return edges 

        def kmeans_cluster(img, k):
            data = np.float32(img).reshape((-1, 3))

            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

            ret, label, center = cv2.kmeans(data, k, None, criteria, 
                5, cv2.KMEANS_RANDOM_CENTERS)
            center = np.uint8(center)
            result = center[label.flatten()]
            result = result.reshape(img.shape)
            return result
        
        @widgets.interact_manual(ksize = (3, 25, 2), 
        block_size = (1, 21, 2), k = (1, 20, 1), d = (5, 15, 2), 
        sigmacolor = (1, 300),sigmaspace = (1, 300))

        def cartoon_image(ksize = 5, 
        block_size = 7, k = 7, d = 7, sigmacolor = 200, sigmaspace = 200):
            edge_Mask = edge_mask(img, ksize, block_size)
            cluster_img = kmeans_cluster(img, k)
            bilateral = cv2.bilateralFilter(cluster_img, d=d, 
            sigmaColor=sigmacolor, sigmaSpace=sigmaspace)
            cartoon = cv2.bitwise_and(bilateral, bilateral, mask = edge_Mask)

            plt.figure(figsize = (15, 8))
            plt.subplot(1, 2, 1)
            plt.yticks([]), plt.xticks([])
            plt.title("Original Image")
            plt.imshow(img)
            plt.subplot(1, 2, 2)
            plt.yticks([]), plt.xticks([])
            plt.title("Cartoon Image")
            plt.imshow(cartoon)
            plt.show()
            
            
# https://github.com/junseongwoo/Image_to_Cartoon_Image

interactive(children=(Dropdown(description='style', index=1, options=('detail', 'pencil', 'cartoon'), value='p…

# 기하학적 변환 (Geometric transformation)

## 이동변환 (translate)

In [69]:
import sys
import numpy as np
import cv2


src = cv2.imread('Ch_2_Opencv_basic/fig/puppy.bmp')

if src is None:
    print('Image load failed!')
    sys.exit()

affine = np.array([[2, 1, 30],
                  [0, 1.5, 50]], np.float32)

dst = cv2.warpAffine(src, affine, (1000, 800))
# dst = cv2.warpAffine(src, affine, (0, 0),
#                      borderMode = cv2.BORDER_CONSTANT,
#                     borderValue = (255, 255, 255))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [72]:
import sys
import numpy as np
import cv2

src = cv2.imread('Ch_6_geometric_trans/fig/rose.jpg')

dst = cv2.resize(src, (600, 500),
                interpolation = cv2.INTER_LINEAR)
dst2 = cv2.resize(src, (600, 500),
                interpolation = cv2.INTER_CUBIC)

if src is None:
    print('Image load failed!')
    sys.exit()
    
cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.imshow('dst2', dst2)

cv2.waitKey()
cv2.destroyAllWindows()

## 회전변환 (Rotation)

In [74]:
import sys
import numpy as np
import cv2

src = cv2.imread('Ch_6_geometric_trans/fig/puppy.bmp')
if src is None:
    print('Image load failed!')
    sys.exit()
    
h, w = src.shape[:2]
rot_affine = cv2.getRotationMatrix2D((w//2, h//2), 20, 1)

dst = cv2.warpAffine(src, rot_affine, (0, 0))
    
cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [45]:
!pip install ipywidgets

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [46]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [51]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np

def f(k):
    x = np.linspace(0, 10, num=1000)
    y = np.sin(k*x)
    plt.plot(x, y)
    plt.show()
    
interact(f, k=(0.0,5.0,0.1) )

interactive(children=(FloatSlider(value=2.5, description='k', max=5.0), Output()), _dom_classes=('widget-inter…

<function __main__.f(k)>

In [75]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np

x = np.linspace(0, 2*np.pi, num=5000)
y = []
param = 50

for i in range(1,param+1):
    y.append([np.sin(i*j) for j in x])
    
def f(k):
    plt.plot(x, y[k])
    plt.show()
    
interact(f, k=(0,param-1) )

interactive(children=(IntSlider(value=24, description='k', max=49), Output()), _dom_classes=('widget-interact'…

<function __main__.f(k)>

In [66]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np

x = np.linspace(0, 2*np.pi, num=5000)
y1, y2 = [], []

param = 50

for i in range(1,param+1):
    y1.append([np.sin(i*j) for j in x])
    y2.append([np.cos(i*j)+np.cos(j) for j in x])
    
@interact(k=(0,param-1), mode=['A','B', 'C'])
def f(k,mode):
    if mode=='A':
        plt.plot(x, y1[k])
    elif mode=='B':
        plt.plot(x, y2[k])
    else:
        print('Error')
        
    plt.show()

interactive(children=(IntSlider(value=24, description='k', max=49), Dropdown(description='mode', options=('A',…

In [67]:
x = np.linspace(0, 2*np.pi, num=5000)
y, y1, y2, y3 = [], [], [], []

param = 50

for i in range(1,param+1):
    y1.append([np.sin(i*j) for j in x])
    y2.append([np.cos(i*j)+np.cos(j) for j in x])
    y3.append([np.tan(i*j)+np.cos(j) for j in x])

y = [y1, y2, y3]

# 그래프를 생성
@interact(k=(0,param-1))
def f(k):
    fig = plt.figure(figsize=(17,4))

    # 타이틀을 배열에 저장
    title=['title1','title2','title3']    

    # 그래프 간의 폭을 조정
    plt.subplots_adjust(wspace=0.25, hspace=0.6)

    for i in range(3):
        ax = fig.add_subplot(1, 3, i+1)
        ax.plot(x, y[i][k])
        ax.set_xlabel('x-axis label') # x축의 라벨
        ax.set_ylabel('y-axis label') # y축의 라벨
        ax.set_title(title[i]) # 타이틀

    plt.show()

interactive(children=(IntSlider(value=24, description='k', max=49), Output()), _dom_classes=('widget-interact'…

In [68]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np

x = np.linspace(0, 2*np.pi, num=5000)
y, y1, y2, y3 = [], [], [], []

param = 50

for i in range(1,param+1):
    y1.append([np.sin(i*j) for j in x])
    y2.append([np.cos(i*j)+np.cos(j) for j in x])
    y3.append([np.tan(i*j)+np.cos(j) for j in x])

y = [y1, y2, y3]

# 그래프를 생성한다.
@interact(k=(0,param-1))
def f(k):
    fig = plt.figure(figsize=(10,10))

    # 타이틀을 배열에 저장한다.
    title=['title1','title2','title3']    

    # 그래프 간의 폭을 조정한다.
    plt.subplots_adjust(wspace=0.2, hspace=0.6)

    for i in range(3):
        ax = fig.add_subplot(3, 1, i+1)
        ax.plot(x, y[i][k])
        ax.set_xlabel('x-axis label') # x축의 라벨
        ax.set_ylabel('y-axis label') # y축의 라벨
        ax.set_title(title[i]) # 타이틀

    plt.show()

interactive(children=(IntSlider(value=24, description='k', max=49), Output()), _dom_classes=('widget-interact'…

In [71]:
import numpy as np
import pandas as pd
from sklearn import datasets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
# import cufflinks as cf
# cf.go_offline(connected=True)

df = pd.DataFrame(np.random.random([100,3]) * 10)
df.columns = ['Feature1','Feature2','Feature3']
df.head()

,Feature1,Feature2,Feature3
0,6.554601,1.429141,8.007380
1,9.160206,2.997878,1.796959
2,8.271171,3.152951,6.801186
3,5.493448,5.327233,0.324195
4,8.024908,2.407495,2.322392


In [73]:
@interact
def show_data_more_than(column=['Feature1', 'Feature2','Feature3'], 
                        x=(0,10,1)):
    return df.loc[df[column] > x]

interactive(children=(Dropdown(description='column', options=('Feature1', 'Feature2', 'Feature3'), value='Feat…